# Accessibility Improvements for the Disabled

In [3]:
#importing dependencies
import speech_recognition as sr
import time
from gtts import gTTS
import os

In [4]:
def text_to_speech(text):
    # Initialize gTTS with the text to convert
    speech = gTTS(text)

    # Save the audio file to a temporary file
    speech_file = 'speech.mp3'
    speech.save(speech_file)

    # Play the audio file
    os.system('afplay ' + speech_file)

# text_to_speech('Hello, world! This is a test.')

In [5]:
recognizer = sr.Recognizer()

In [6]:
def capture_voice_input():
    with sr.Microphone() as source:
        print("Listening...")
        recognizer.adjust_for_ambient_noise(source)

        try:
            audio = recognizer.listen(source, timeout=5, phrase_time_limit=5)
        except sr.WaitTimeoutError:
            print("Listening timed out while waiting for you to speak")
            return None
    return audio

In [7]:
def convert_voice_to_text(audio):
    if audio is None:
        return ""
    try:
        text = recognizer.recognize_google(audio)
        print("You said: " + text)
    except sr.UnknownValueError:
        text = ""
        print("Sorry I didn't understand that.")
    except sr.RequestError as e:
        text = ""
        print("Error: {0}".format(e))
    return text

In [8]:
def process_voice_command(text):
    if "hello" in text.lower():
        print("Hello! How can I help you?")
    elif "what is your name" in text.lower():
        print("My name is Eden")
    elif ("how are you doing today" in text.lower() or
          "how are you" in text.lower() or
          "how are you doing" in text.lower()):
        print("I'm doing alright, thank you sir")
    elif ("alright, goodbye" in text.lower() or 
          "alright" in text.lower() or 
          "goodbye" in text.lower() or 
          "all right" in text.lower() or 
          "stop" in text.lower()):
        print("Goodbye! Have a nice day")
        return True
    else: print("I didn't understand that command. Please try again.")
    return False

In [9]:
def main():
    end_program = False
    attempts = 0
    max_attempts = 3  # Limit to the number of attempts

    while not end_program and attempts < max_attempts:
        audio = capture_voice_input()
        if audio is None:
            attempts += 1
            print(f"Retrying... ({attempts}/{max_attempts})")
            time.sleep(1)  # Delay to prevent rapid looping
        else:
            text = convert_voice_to_text(audio)
            if text == "":
                attempts += 1
                print(f"Retrying... ({attempts}/{max_attempts})")
            else:
                end_program = process_voice_command(text)
                attempts = 0  # Reset attempts if a valid audio input is processed
    if attempts >= max_attempts:
        print("Too many failed attempts due to timeout. Exiting program.")

if __name__ == "__main__":
    main()


Listening...
You said: hello how are you doing today
Hello! How can I help you?
Listening...
You said: how are you doing
I'm doing alright, thank you sir
Listening...
Listening timed out while waiting for you to speak
Retrying... (1/3)
Listening...
Sorry I didn't understand that.
Retrying... (2/3)
Listening...
Listening timed out while waiting for you to speak
Retrying... (3/3)
Too many failed attempts due to timeout. Exiting program.
